# Demanda de Helado — Preparación de Datos para XGBoost  
*(modelos independientes por sabor)*  

## 1 · Objetivo  
Transformar la serie diaria de producción de helado y variables exógenas (clima, calendario, etc.) en un **dataset tabular** que XGBoost pueda consumir, listo para predecir los **próximos 14 días** de producción **por sabor**.

---

## 2 · Datos crudos (formato real)

En los archivos originales **cada sabor es una columna**:  

| fecha      | temp_min | temp_max | lluvia | feriado | chocolate | vainilla | frutilla |
|------------|----------|----------|--------|---------|-----------|----------|----------|
| 2025‑04‑01 | 11       | 25       | 0      | 0       | 520       | 480      | 460      |
| 2025‑04‑02 | 12       | 26       | 1      | 0       | 510       | 490      | 455      |
| 2025‑04‑03 | 11       | 27       | 0      | 0       | 525       | 500      | 470      |
| …          | …        | …        | …      | …       | …         | …        | …        |

* Una fila por **día**.  
* Las columnas de sabores (`chocolate`, `vainilla`, `frutilla`, …) contienen los kg producidos.  

---

## 3 · Estrategia de preparación

Para **cada sabor** se hará:

1. **Separar la columna del sabor** como serie‑target → `kg`.  
2. **Lags** de `kg` (1, 2, 3, 7, 14 días).  
3. **Promedio móvil** de 7 días (`kg_roll7_mean`).  
4. **Targets adelantados** `kg_t+1 … kg_t+14`.  
5. Eliminar filas con `NaN` generados por los desplazamientos.

Ejemplo de la tabla resultante para *chocolate*:

| fecha      | temp_min | temp_max | lluvia | feriado | kg_lag1 | kg_lag7 | kg_roll7_mean | kg_t+1 | … | kg_t+14 |
|------------|----------|----------|--------|---------|---------|---------|---------------|--------|---|---------|
| 2025‑04‑15 | 13       | 27       | 0      | 0       | 525     | 515     | 518.4         | 530    | … | 560     |
| 2025‑04‑16 | 12       | 26       | 1      | 0       | 530     | 510     | 519.7         | 540    | … | 555     |

> *Repetir exactamente el mismo proceso con la columna de vainilla, frutilla, etc.*

---

## 4 · Separación train / test y convención de nombres

Se entrenará **un modelo por sabor**:

```python
# Ejemplo para chocolate
X_train_choc, y_train_choc = …
X_test_choc,  y_test_choc  = …

# Ejemplo para vainilla
X_train_vain, y_train_vain = …
X_test_vain,  y_test_vain  = …
